In [ ]:
import numpy as np
import pandas as pd
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, SpatialDropout1D
from keras.layers import LSTM
from keras.datasets import imdb, reuters
from keras.utils import to_categorical
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
max_words = 20_000
line_length = 200
batch_size = 32

In [ ]:
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words)

In [ ]:
# number of articles
x_train.shape, y_train.shape

In [ ]:
# reuters categories
np.unique(y_train)

In [ ]:
# mapping dictionaries
word_to_id = reuters.get_word_index()
id_to_word = {v:k for k,v in word_to_id.items()}

In [ ]:
id_to_word[0] = '<START>'

In [ ]:
def article(index):
    return ' '.join([id_to_word[max(0, idx-3)] for idx in x_train[index]])

In [ ]:
for i in range(10):
    print(article(i), '\n\n\n')

In [ ]:
# make all rows the same length
x_train = sequence.pad_sequences(x_train, maxlen=line_length)
x_test = sequence.pad_sequences(x_test, maxlen=line_length)

In [ ]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
x_train.shape, y_train.shape

In [ ]:
model = Sequential()
model.add(Embedding(max_words, 128))
model.add(LSTM(128, dropout=0.5, recurrent_dropout=0.5))
model.add(Dense(46, activation='softmax'))
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# ************************************************
# CHANGE THE EPOCHS, BELOW, TO GET HIGHER ACCURACY
# ************************************************

history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=2,
          validation_data=(x_test, y_test))

In [ ]:
fig, ax1 = plt.subplots(1,1,figsize=(12,7))
ax1.plot(history.epoch, history.history['loss'], marker='^', color='purple')
ax1.set_xlabel('epochs')
ax1.set_ylabel('loss', color='purple')
ax1.tick_params('y', colors='purple')

ax2 = ax1.twinx()
plt.plot(history.epoch, history.history['acc'], marker='+', color='green', label='train')
ax2.set_ylim(0,1)

ax3 = ax1.twinx()
plt.plot(history.epoch, history.history['val_acc'], marker='*', color='red', label='validation')
ax3.set_ylim(0,1)

fig.suptitle('classifying reuters articles');
fig.legend();